# Adatbeolvasás

In [ ]:
# 📦 Szükséges csomagok importálása
import pandas as pd
import numpy as np

print("Csomagok betöltve, készen állunk!")


In [ ]:
# 📂 CSV fájlok helye
csv_path = "/home/g14/sze_adatelemzes/danci/database"

# CSV fájlok beolvasása
versenyzok = pd.read_csv(f"{csv_path}/f1_dataset_versenyzok.csv")
konstruktorok = pd.read_csv(f"{csv_path}/f1_dataset_konstruktorok.csv")
helyszinek = pd.read_csv(f"{csv_path}/f1_dataset_helyszinek.csv")
idojarasi_korulmenyek = pd.read_csv(f"{csv_path}/f1_dataset_idojarasi_korulmenyek.csv")
autok = pd.read_csv(f"{csv_path}/f1_dataset_autok.csv")
szezonok = pd.read_csv(f"{csv_path}/f1_dataset_szezonok.csv")
futamok = pd.read_csv(f"{csv_path}/f1_dataset_futamok.csv")
helyezesek = pd.read_csv(f"{csv_path}/f1_dataset_helyezesek.csv")

print("CSV fájlok sikeresen betöltve!")


In [ ]:
# 📊 Versenyzők táblázat előnézete
display(versenyzok.head())

# Típuskonverziók

In [ ]:
# 🔄 Adattípus konverziók

#  A 'szuletesi_datum' oszlopot datetime típusra konvertáljuk.
#  Ha az érték nem konvertálható, NaT lesz (errors="coerce").
versenyzok["szuletesi_datum"] = pd.to_datetime(
    versenyzok["szuletesi_datum"], errors="coerce"
)

#  A 'datum' oszlopot datetime típusú dátummá konvertáljuk.
#  Csak a dátumot tartjuk meg (.dt.date), az időt eldobjuk.
futamok["datum"] = pd.to_datetime(futamok["datum"], errors="coerce").dt.date

#  Az 'ido' oszlopot datetime típusú idővé konvertáljuk a "%H:%M:%S" formátum szerint.
#  Csak az időt tartjuk meg (.dt.time).
futamok["ido"] = pd.to_datetime(
    futamok["ido"], format="%H:%M:%S", errors="coerce"
).dt.time

#  A 'ev' oszlopot datetime formátumra konvertáljuk, csak az év alapján.
#  Ha csak az év szerepel, január 1. lesz automatikusan hozzárendelve.
szezonok["ev"] = pd.to_datetime(
    szezonok["ev"], format="%Y", errors="coerce"
)

#  A 'verseny_ido' oszlopot datetime típusú idővé konvertáljuk a "%H:%M:%S" formátum szerint.
#  Csak az időt tartjuk meg (.dt.time).
helyezesek["verseny_ido"] = pd.to_datetime(
    helyezesek["verseny_ido"], format="%H:%M:%S", errors="coerce"
).dt.time

# Kiírás a felhasználónak, hogy minden konverzió sikeresen végrehajtva
print("Adattípus konverziók sikeresen végrehajtva!")


# Formátum ellenőrzés

In [ ]:
# ✅ Ellenőrzés: adattípusok megtekintése
print("Versenyzők:\n", versenyzok.dtypes, "\n")
print("Futamok:\n", futamok.dtypes, "\n")
print("Szezonok:\n", szezonok.dtypes, "\n")
print("Helyezések:\n", helyezesek.dtypes, "\n")


# Statisztikai jellemzők

In [ ]:
# Listázunk minden DataFrame-et egy szótárban
dfs = {
    "versenyzok": versenyzok,
    "konstruktorok": konstruktorok,
    "helyszinek": helyszinek,
    "idojarasi_korulmenyek": idojarasi_korulmenyek,
    "autok": autok,
    "szezonok": szezonok,
    "futamok": futamok,
    "helyezesek": helyezesek
}

# Minden DataFrame statisztikai jellemzőinek kiíratása
for name, df in dfs.items():
    print(f"Tábla neve: {name}")
    print(f"Sorok és oszlopok: {df.shape}")
    print("\n")

# Lekérdezések

## 1. Listázza ki azokat a pilótákat, akik pole pozícióból kerültek dobogós helyre a 2019-es szezonban!

In [ ]:
# Merge: helyezesek + szezonok (2019-re szűrés)
merged = helyezesek.merge(szezonok, on="szezon_id")

# Oszlopok ellenőrzése
print(merged.columns.tolist())

# Szűrés: 2019, dobogó, pole
filtered = merged[
    (merged["ev"].dt.year == 2019) &   # szezon 2019
    (merged["elert_helyezes"] <= 3) &  # dobogó
    (merged["pole_poz"] == 1)          # pole pozíció
]

# Merge (összekapcsolás) a 'filtered' DataFrame és a 'versenyzok' DataFrame között
# a 'versenyzo_id' oszlop alapján, hogy megkapjuk a versenyzők nevét
result = filtered.merge(versenyzok, on="versenyzo_id")

# Új oszlop létrehozása 'Nev' néven, ahol a kereszt- és vezetéknév össze van fűzve
result["Nev"] = result["keresztnev"] + " " + result["vezeteknev"]

# Csak a különböző neveket hagyjuk meg (DISTINCT)
# A reset_index(drop=True) eltávolítja a régi indexet, újrakezdi 0-tól
pilota_lista = result["Nev"].drop_duplicates().reset_index(drop=True)

# 4. Az eredmény megjelenítése
display(pilota_lista)



## 2. Listázza ki azon autókat, melyeknek teljesítménye 700 és 800 kW közé esik!


In [ ]:
# Győződjünk meg róla, hogy a teljesítmény numerikus
autok["teljesitmeny"] = pd.to_numeric(autok["teljesitmeny"], errors="coerce")

# Szűrés: teljesítmény 700–800 kW
filtered_autok = autok[(autok["teljesitmeny"] >= 700) & (autok["teljesitmeny"] <= 800)]

# Csak az oszlopok, amik érdekelnek, és rendezés csökkenő sorrendben
result = (
    # Kiválasztjuk a 'nev' és 'teljesitmeny' oszlopokat a 'filtered_autok' DataFrame-ből
    filtered_autok[["nev", "teljesitmeny"]]
    
    # Sorba rendezzük a DataFrame-et a 'teljesitmeny' oszlop szerint csökkenő sorrendben
    .sort_values(by="teljesitmeny", ascending=False)
    
    # Újraindexeljük a DataFrame-et 0-tól kezdődő sorszámozással, a régi indexet eldobva
    .reset_index(drop=True)
)

display(result)


## 3. Ki volt a legeredményesebb konstruktőr csapatfőnöke a 2003 és 2007 közötti versenyszezonok intervallumában?

In [ ]:
# Győződjünk meg róla, hogy az 'ev' oszlop datetime
szezonok["ev"] = pd.to_datetime(szezonok["ev"], errors="coerce")

# Merge helyezesek + szezonok
merged = helyezesek.merge(szezonok, on="szezon_id")

# Oszlopok ellenőrzése
print(merged.columns.tolist())

# Szűrés: 2003-2007 közötti szezonok és első helyezés
filtered = merged[
    (merged["ev"].dt.year >= 2003) &
    (merged["ev"].dt.year <= 2007) &
    (merged["elert_helyezes"] == 1)
]

# Merge konstruktorok a csapatvezető nevéhez
merged_final = filtered.merge(
    konstruktorok[["konstruktor_id", "csapatvezeto"]],
    on="konstruktor_id"
)

# Csoportosítás és számolás
result = (
    # Csoportosítás a 'merged_final' DataFrame-ben a 'csapatvezeto' oszlop szerint
    # Ez a konstruktor csapatfőnök neve
    merged_final.groupby("csapatvezeto")["elert_helyezes"]
    
    # Megszámolja az egyes csapatfőnökökhöz tartozó sorok számát,
    # vagyis hány versenyt nyertek (minden első helyezés)
    .count()
    
    # Az eredményt átalakítjuk DataFrame-é, és az új oszlopot 'db'-nek nevezzük
    .reset_index(name="db")
    
    # Csökkenő sorrendbe rendezzük a győzelmek számát
    .sort_values(by="db", ascending=False)
    
    # Csak az első sort tartjuk meg, ami a legeredményesebb csapatfőnök
    .head(1)
)

# Megjelenítjük az eredményt
display(result)



## 4. Számítsa ki a különböző időjárási viszonyok között rendezett futamok számát!

In [ ]:
# Merge futamok és idojarasi_korulmenyek a nev oszlophoz
merged = futamok.merge(
    idojarasi_korulmenyek[["idojarasi_korulmeny_id", "nev"]],
    on="idojarasi_korulmeny_id"
)

# Oszlopok ellenőrzése
print(merged.columns.tolist())

# Csoportosítás és számolás
result = (
    # Csoportosítás a 'merged' DataFrame-ben a 'nev' oszlop szerint
    # Ez az időjárási viszony neve (pl. 'paras', 'esos', stb.)
    merged.groupby("nev")["idojarasi_korulmeny_id"]
    
    # Megszámolja az egyes csoportokban hány sor van,
    # vagyis hány futamot tartalmaz az adott időjárási viszony
    .count()
    
    # Az eredményt átalakítjuk DataFrame-é, és az új oszlopot 'db'-nek nevezzük
    .reset_index(name="db")
    
    # Csökkenő sorrendbe rendezzük az időjárási viszonyok szerinti futamszámot
    .sort_values(by="db", ascending=False)
)

# Megjelenítjük az eredményt
display(result)



## 5. Melyik 5, a konstruktőrök alapítási helye szerinti ország rendelkezik a legtöbb győzelemmel?


In [ ]:
# Szűrés: csak az 1. helyezések
wins = helyezesek[helyezesek["elert_helyezes"] == 1]

# Merge a konstruktorokkal, hogy megkapjuk az országot
merged = wins.merge(konstruktorok[["konstruktor_id", "orszag"]], on="konstruktor_id")

# Oszlopok ellenőrzése
print(merged.columns.tolist())

# Győzelmek száma országonként
result = (
    # Csoportosítjuk a 'merged' DataFrame-et az 'orszag' oszlop szerint
    merged.groupby("orszag")
    
    # Minden csoportban megszámoljuk a sorok számát (itt ez a győzelmek száma)
    .size()
    
    # Átalakítjuk DataFrame-é, és az új oszlopot elnevezzük 'Gyozelmek_szama'-nak
    .reset_index(name="Gyozelmek_szama")
    
    # Csökkenő sorrendbe rendezzük a győzelmek száma szerint
    .sort_values("Gyozelmek_szama", ascending=False)
    
    # Csak az első 5 sort tartjuk meg (legtöbb győzelem)
    .head(5)
)

# Eredmény megjelenítése
display(result)
